In [5]:
#for fastai.structured
!pip install fastai==0.7.0 

     |████████████████████████████████| 122kB 1.8MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 6.2MB/s eta 0:00:01
     |████████████████████████████████| 112kB 13.7MB/s eta 0:00:01
     |████████████████████████████████| 122kB 18.5MB/s eta 0:00:01
     |████████████████████████████████| 3.6MB 8.5MB/s eta 0:00:01
     |████████████████████████████████| 2.2MB 23.7MB/s eta 0:00:01
     |████████████████████████████████| 71kB 7.1MB/s eta 0:00:011
    ERROR: Complete output from command python setup.py egg_info:
    ERROR: Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-install-p61op430/torch/setup.py", line 11, in <module>
        raise RuntimeError(README)
    RuntimeError: PyTorch does not currently provide packages for PyPI (see status at https://github.com/pytorch/pytorch/issues/566).
    
    Please follow the instructions at http://pytorch.org/ to install with miniconda instead.
    
    ----------------------------

In [22]:
os.getcwd()

'/home/dc/TDS'

In [26]:
#!wget http://files.fast.ai/part2/lesson14/rossmann.tgz
import os
from os import path


if not path.exists("rossmann_data"):
    !mkdir rossmann_data
    !wget -P rossmann_data/ http://files.fast.ai/part2/lesson14/rossmann.tgz
    !tar xvf rossmann_data/rossmann.tgz -C rossmann_data


#verify we have files
if not path.exists("rossmann_data/train.csv"):
    print("train.csv not here")
    
print('rossmann download and untar complete')

--2019-06-04 18:20:36--  http://files.fast.ai/part2/lesson14/rossmann.tgz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7730448 (7.4M) [text/plain]
Saving to: ‘rossmann_data/rossmann.tgz’

rossmann.tgz        100%[===================>]   7.37M   165KB/s    in 62s     

2019-06-04 18:21:38 (122 KB/s) - ‘rossmann_data/rossmann.tgz’ saved [7730448/7730448]

googletrend.csv
sample_submission.csv
state_names.csv
store.csv
store_states.csv
test.csv
train.csv
weather.csv


In [154]:
#rossmann
import numpy as np
import pandas as pd
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import fastai
from isoweek import Week
from pandas_summary import DataFrameSummary
#from fastai.structured import *
#from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)
import math, keras, datetime, pandas as pd, numpy as np, keras.backend as K
import matplotlib.pyplot as plt, xgboost, operator, random, pickle

PATH='rossmann_data/'


In [15]:
def concat_csvs(dirname):
    path = f'{PATH}{dirname}'
    filenames=glob(f"{PATH}/*.csv")
 
    wrote_header = False
    with open(f"{path}.csv","w") as outputfile:
        for filename in filenames:
            name = filename.split(".")[0]
            with open(filename) as f:
                line = f.readline()
                if not wrote_header:
                    wrote_header = True
                    outputfile.write("file,"+line)
                for line in f:
                     outputfile.write(name + "," + line)
                outputfile.write("\n")

In [16]:
table_names = ['train', 'store', 'store_states', 'state_names', 
               'googletrend', 'weather', 'test']

In [27]:

tables = [pd.read_csv("rossmann_data/"+fname+'.csv', low_memory=False) for fname in table_names]

In [30]:
from IPython.display import HTML
for t in tables: display(t.head())


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN


,StateName,State
0,BadenWuerttemberg,BW
1,Bayern,BY
2,Berlin,BE
3,Brandenburg,BB
4,Bremen,HB


,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [32]:
train, store, store_states, state_names, googletrend, weather, test = tables

In [33]:
len(test), len(train)

(41088, 1017209)

In [34]:
train.StateHoliday = train.StateHoliday!='0'
test.StateHoliday = test.StateHoliday!='0'

In [35]:
def join_df(left, right, left_on, right_on=None):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", "_y"))

In [36]:
weather = join_df(weather, state_names, "file", "StateName")

In [37]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'

In [106]:
def add_datepart(df):
    df.Date = pd.to_datetime(df.Date)
    df["Year"] = df.Date.dt.year
    df["Month"] = df.Date.dt.month
    df["Week"] = df.Date.dt.week
    df["Day"] = df.Date.dt.day

In [107]:
add_datepart(weather)
add_datepart(googletrend)
add_datepart(train)
add_datepart(test)


In [108]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

In [109]:
store = join_df(store, store_states, "Store")
len(store[store.State.isnull()])

0

In [42]:
joined = join_df(train, store, "Store")
len(joined[joined.StoreType.isnull()])

0

In [43]:
joined = join_df(joined, googletrend, ["State","Year", "Week"])
len(joined[joined.trend.isnull()])

0

In [44]:
joined = joined.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined[joined.trend_DE.isnull()])

0

In [45]:
joined = join_df(joined, weather, ["State","Date"])
len(joined[joined.Mean_TemperatureC.isnull()])

0

In [46]:
joined_test = test.merge(store, how='left', left_on='Store', right_index=True)
len(joined_test[joined_test.StoreType.isnull()])

48

In [59]:
joined.CompetitionOpenSinceYear = joined.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
joined.CompetitionOpenSinceMonth = joined.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
joined.Promo2SinceYear = joined.Promo2SinceYear.fillna(1900).astype(np.int32)
joined.Promo2SinceWeek = joined.Promo2SinceWeek.fillna(1).astype(np.int32)

In [63]:
print(len(joined.CompetitionOpenSinceYear))
print(len(joined.CompetitionOpenSinceMonth))
print(len(joined.Promo2SinceYear))
print(len(joined.Promo2SinceWeek))

1017209
1017209
1017209
1017209


In [92]:

joined.CompetitionOpenSinceYear.head()
print(type(joined.CompetitionOpenSinceYear))
# a series data type is a column or nd.array which as been assigned in index!
#it can be accessed like an array
print(joined.CompetitionOpenSinceYear[0])


<class 'pandas.core.series.Series'>
2008


In [100]:
#a pandas date_time what is a date time format?
import datetime
one =datetime.datetime(2018, 1, 1)
print(type(one))
print(one) #ah default 0,0,0 for hours, mins secs. 
two = pd.to_datetime(datetime.datetime(2018, 8,15))
print(type(two),two)
#what does joined.apply do? to a dataframe? 
print(type(joined))
# what is the format we have in the different columns? and what do we want to convert to? 
#len(joined.CompetitionOpenSinceYear)
for x in range(10):
    print(datetime.datetime(joined.CompetitionOpenSinceYear[x],joined.CompetitionOpenSinceMonth[x],15))
    #print(datetime.datetime(joined.CompetitionOpenSinceYear[x]))

<class 'datetime.datetime'>
2018-01-01 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'> 2018-08-15 00:00:00
<class 'pandas.core.frame.DataFrame'>
2008-09-15 00:00:00
2007-11-15 00:00:00
2006-12-15 00:00:00
2009-09-15 00:00:00
2015-04-15 00:00:00
2013-12-15 00:00:00
2013-04-15 00:00:00
2014-10-15 00:00:00
2000-08-15 00:00:00
2009-09-15 00:00:00


In [101]:
#removed astype(pd.datetime))
joined["CompetitionOpenSince"] = pd.to_datetime(joined.apply(lambda x: datetime.datetime(
    x.CompetitionOpenSinceYear, x.CompetitionOpenSinceMonth, 15), axis=1))

#.astype(pd.datetime))


In [102]:
joined["CompetitionDaysOpen"] = joined.Date.subtract(joined["CompetitionOpenSince"]).dt.days

In [103]:
joined.loc[joined.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
joined.loc[joined.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

In [104]:
joined["CompetitionMonthsOpen"] = joined["CompetitionDaysOpen"]//30
joined.loc[joined.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined.CompetitionMonthsOpen.unique()

array([24,  3, 19,  9,  0, 16, 17,  7, 15, 22, 11, 13,  2, 23, 12,  4, 10,
        1, 14, 20,  8, 18,  6, 21,  5])

In [114]:
#.astype(pd.datetime)
joined["Promo2Since"] = pd.to_datetime(joined.apply(lambda x: Week(
    x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1))
joined["Promo2Days"] = joined.Date.subtract(joined["Promo2Since"]).dt.days


In [115]:
joined.loc[joined.Promo2Days<0, "Promo2Days"] = 0
joined.loc[joined.Promo2SinceYear<1990, "Promo2Days"] = 0

In [116]:
joined["Promo2Weeks"] = joined["Promo2Days"]//7
joined.loc[joined.Promo2Weeks<0, "Promo2Weeks"] = 0
joined.loc[joined.Promo2Weeks>25, "Promo2Weeks"] = 25
joined.Promo2Weeks.unique()

array([ 0, 25, 17,  8, 13, 24, 16,  7, 12, 23, 15,  6, 11, 22, 14,  5, 10,
       21,  4,  9, 20,  3, 19,  2, 18,  1])

In [117]:
columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]


In [118]:
class elapsed(object):
    def __init__(self, fld):
        self.fld = fld
        self.last = pd.to_datetime(np.nan)
        self.last_store = 0
        
    def get(self, row):
        if row.Store != self.last_store:
            self.last = pd.to_datetime(np.nan)
            self.last_store = row.Store
        if (row[self.fld]): self.last = row.Date
        return row.Date-self.last

In [119]:
df = train[columns]

In [120]:
def add_elapsed(fld, prefix):
    sh_el = elapsed(fld)
    df[prefix+fld] = df.apply(sh_el.get, axis=1)

In [121]:
fld = 'SchoolHoliday'
df = df.sort_values(['Store', 'Date'])
add_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
add_elapsed(fld, 'Before')

In [122]:
fld = 'StateHoliday'
df = df.sort_values(['Store', 'Date'])
add_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
add_elapsed(fld, 'Before')

In [123]:
fld = 'Promo'
df = df.sort_values(['Store', 'Date'])
add_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
add_elapsed(fld, 'Before')


In [124]:
df = df.set_index("Date")

In [125]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']

In [126]:
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(pd.Timedelta(0)).dt.days

In [127]:
bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()

In [128]:
fwd = df[['Store']+columns].sort_index(ascending=False
                                      ).groupby("Store").rolling(7, min_periods=1).sum()

In [129]:
bwd.drop('Store',1,inplace=True)
bwd.reset_index(inplace=True)

In [130]:
fwd.drop('Store',1,inplace=True)
fwd.reset_index(inplace=True)

In [131]:
df.reset_index(inplace=True)

In [132]:
df = df.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
df = df.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])

In [133]:
df.drop(columns,1,inplace=True)
df.head()


,Date,Store,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,2015-07-31,1,0,0,57,0,0,0,5.0,0.0,5.0,1.0,0.0,1.0
1,2015-07-30,1,0,0,56,0,0,0,4.0,0.0,4.0,2.0,0.0,2.0
2,2015-07-29,1,0,0,55,0,0,0,3.0,0.0,3.0,3.0,0.0,3.0
3,2015-07-28,1,0,0,54,0,0,0,2.0,0.0,2.0,4.0,0.0,4.0
4,2015-07-27,1,0,0,53,0,0,0,1.0,0.0,1.0,5.0,0.0,5.0


In [134]:
df.to_csv('df.csv')

In [135]:
df = pd.read_csv('df.csv', index_col=0)

/home/dc/anaconda3/envs/TDS/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [140]:
df["Date"] = pd.to_datetime(df.Date)

In [141]:
df.columns

Index(['Date', 'Store', 'AfterSchoolHoliday', 'BeforeSchoolHoliday',
       'AfterStateHoliday', 'BeforeStateHoliday', 'AfterPromo', 'BeforePromo',
       'SchoolHoliday_bw', 'StateHoliday_bw', 'Promo_bw', 'SchoolHoliday_fw',
       'StateHoliday_fw', 'Promo_fw'],
      dtype='object')

In [142]:
joined = join_df(joined, df, ['Store', 'Date'])

In [143]:
df = df.set_index("Date")

In [144]:
joined = join_df(joined, df, ['Store', 'Date'])

In [145]:
joined.to_csv('joined.csv')

In [146]:
joined = pd.read_csv('joined.csv', index_col=0)
joined["Date"] = pd.to_datetime(joined.Date)
joined.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year',
       ...
       'AfterStateHoliday_y.2', 'BeforeStateHoliday_y.2', 'AfterPromo_y.2',
       'BeforePromo_y.2', 'SchoolHoliday_bw_y.2', 'StateHoliday_bw_y.2',
       'Promo_bw_y.2', 'SchoolHoliday_fw_y.2', 'StateHoliday_fw_y.2',
       'Promo_fw_y.2'],
      dtype='object', length=118)

In [148]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler

In [149]:
cat_var_dict = {'Store': 50, 'DayOfWeek': 6, 'Year': 2, 'Month': 6,
'Day': 10, 'StateHoliday': 3, 'CompetitionMonthsOpen': 2,
'Promo2Weeks': 1, 'StoreType': 2, 'Assortment': 3, 'PromoInterval': 3,
'CompetitionOpenSinceYear': 4, 'Promo2SinceYear': 4, 'State': 6,
'Week': 2, 'Events': 4, 'Promo_fw': 1,
'Promo_bw': 1, 'StateHoliday_fw': 1,
'StateHoliday_bw': 1, 'SchoolHoliday_fw': 1,
'SchoolHoliday_bw': 1}

In [155]:
cat_vars = [o[0] for o in 
            sorted(cat_var_dict.items(), key=operator.itemgetter(1), reverse=True)]

In [156]:
"""cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday',
           'StoreType', 'Assortment', 'Week', 'Events', 'Promo2SinceYear',
            'CompetitionOpenSinceYear', 'PromoInterval', 'Promo', 'SchoolHoliday', 'State']"""

"cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday',\n           'StoreType', 'Assortment', 'Week', 'Events', 'Promo2SinceYear',\n            'CompetitionOpenSinceYear', 'PromoInterval', 'Promo', 'SchoolHoliday', 'State']"

In [157]:
# mean/max wind; min temp; cloud; min/mean humid; 
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

In [158]:
"""contin_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 
   'Max_Humidity', 'trend', 'trend_DE', 'AfterStateHoliday', 'BeforeStateHoliday']"""

"contin_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', \n   'Max_Humidity', 'trend', 'trend_DE', 'AfterStateHoliday', 'BeforeStateHoliday']"

In [159]:
for v in contin_vars: joined.loc[joined[v].isnull(), v] = 0
for v in cat_vars: joined.loc[joined[v].isnull(), v] = ""

In [160]:
cat_maps = [(o, LabelEncoder()) for o in cat_vars]
contin_maps = [([o], StandardScaler()) for o in contin_vars]

In [161]:
cat_mapper = DataFrameMapper(cat_maps)
cat_map_fit = cat_mapper.fit(joined)
cat_cols = len(cat_map_fit.features)
cat_cols

22

In [162]:
contin_mapper = DataFrameMapper(contin_maps)
contin_map_fit = contin_mapper.fit(joined)
contin_cols = len(contin_map_fit.features)
contin_cols

16

In [163]:
cat_map_fit.transform(joined)[0,:5], contin_map_fit.transform(joined)[0,:5]

(array([ 0, 30,  4,  6,  4]),
 array([-0.53772351,  1.02444552,  0.81863226,  0.32504749,  0.61112004]))

In [164]:
pickle.dump(contin_map_fit, open('contin_maps.pickle', 'wb'))
pickle.dump(cat_map_fit, open('cat_maps.pickle', 'wb'))

In [165]:
[len(o[1].classes_) for o in cat_map_fit.features]

[1115, 31, 7, 12, 12, 23, 8, 22, 2, 3, 4, 3, 25, 4, 52, 26, 6, 6, 3, 3, 8, 8]

In [166]:
joined_sales = joined[joined.Sales!=0]
n = len(joined_sales)

In [167]:
n


844338

In [169]:
samp_size = 100000
np.random.seed(42)
idxs = sorted(np.random.choice(n, samp_size, replace=False))

In [170]:
joined_samp = joined_sales.iloc[idxs].set_index("Date")

In [171]:
samp_size = n
joined_samp = joined_sales.set_index("Date")

In [172]:
train_ratio = 0.9
train_size = int(samp_size * train_ratio)

In [173]:
train_size

759904

In [174]:
joined_valid = joined_samp[train_size:]
joined_train = joined_samp[:train_size]
len(joined_valid), len(joined_train)

(84434, 759904)

In [175]:
def cat_preproc(dat):
    return cat_map_fit.transform(dat).astype(np.int64)

In [176]:
cat_map_train = cat_preproc(joined_train)
cat_map_valid = cat_preproc(joined_valid)

In [177]:
def contin_preproc(dat):
    return contin_map_fit.transform(dat).astype(np.float32)

In [178]:
contin_map_train = contin_preproc(joined_train)
contin_map_valid = contin_preproc(joined_valid)

In [179]:
y_train_orig = joined_train.Sales
y_valid_orig = joined_valid.Sales

In [180]:
max_log_y = np.max(np.log(joined_samp.Sales))
y_train = np.log(y_train_orig)/max_log_y
y_valid = np.log(y_valid_orig)/max_log_y

In [184]:
#y_train = np.log(y_train)
ymean=y_train_orig.mean()
ystd=y_train_orig.std()
y_train = (y_train_orig-ymean)/ystd
#y_valid = np.log(y_valid)
y_valid = (y_valid_orig-ymean)/ystd
print(ymean)
print(ystd)
print(y_train)
print(y_valid)

6992.518589190214
3109.671920399961
Date
2015-07-31   -0.556174
2015-07-31   -0.298591
2015-07-31    0.424958
2015-07-31    2.251839
2015-07-31   -0.697990
2015-07-31   -0.431402
2015-07-31    2.685647
2015-07-31    0.482199
2015-07-31    0.505674
2015-07-31    0.061898
2015-07-31    1.114099
2015-07-31    0.632376
2015-07-31    0.587998
2015-07-31   -0.144233
2015-07-31    0.706982
2015-07-31    1.041422
2015-07-31    0.462261
2015-07-31    0.989970
2015-07-31    0.399232
2015-07-31    0.836256
2015-07-31    0.811173
2015-07-31   -0.137159
2015-07-31    0.090196
2015-07-31    2.314547
2015-07-31    2.311331
2015-07-31    0.265456
2015-07-31    2.000366
2015-07-31    0.099201
2015-07-31    0.777729
2015-07-31   -0.712782
                ...   
2013-04-02   -0.907336
2013-04-02   -0.185074
2013-04-02   -0.473529
2013-04-02   -0.333964
2013-04-02   -0.648788
2013-04-02    0.156763
2013-04-02    0.298900
2013-04-02   -0.599908
2013-04-02    2.205211
2013-04-02   -0.468062
2013-04-02   -0.

In [185]:
def rmspe(y_pred, targ = y_valid_orig):
    pct_var = (targ - y_pred)/targ
    return math.sqrt(np.square(pct_var).mean())

In [186]:
def log_max_inv(preds, mx = max_log_y):
    return np.exp(preds * mx)

In [187]:
def normalize_inv(preds):
    return preds * ystd + ymean

In [188]:
"""
1 97s - loss: 0.0104 - val_loss: 0.0083
2 93s - loss: 0.0076 - val_loss: 0.0076
3 90s - loss: 0.0071 - val_loss: 0.0076
4 90s - loss: 0.0068 - val_loss: 0.0075
5 93s - loss: 0.0066 - val_loss: 0.0075
6 95s - loss: 0.0064 - val_loss: 0.0076
7 98s - loss: 0.0063 - val_loss: 0.0077
8 97s - loss: 0.0062 - val_loss: 0.0075
9 95s - loss: 0.0061 - val_loss: 0.0073
0 101s - loss: 0.0061 - val_loss: 0.0074
"""

'\n1 97s - loss: 0.0104 - val_loss: 0.0083\n2 93s - loss: 0.0076 - val_loss: 0.0076\n3 90s - loss: 0.0071 - val_loss: 0.0076\n4 90s - loss: 0.0068 - val_loss: 0.0075\n5 93s - loss: 0.0066 - val_loss: 0.0075\n6 95s - loss: 0.0064 - val_loss: 0.0076\n7 98s - loss: 0.0063 - val_loss: 0.0077\n8 97s - loss: 0.0062 - val_loss: 0.0075\n9 95s - loss: 0.0061 - val_loss: 0.0073\n0 101s - loss: 0.0061 - val_loss: 0.0074\n'

In [189]:
def split_cols(arr): return np.hsplit(arr,arr.shape[1])

In [190]:
map_train = split_cols(cat_map_train) + [contin_map_train]
map_valid = split_cols(cat_map_valid) + [contin_map_valid]

In [191]:
len(map_train)

23

In [192]:
map_train = split_cols(cat_map_train) + split_cols(contin_map_train)
map_valid = split_cols(cat_map_valid) + split_cols(contin_map_valid)

In [193]:
def cat_map_info(feat): return feat[0], len(feat[1].classes_)

In [194]:
cat_map_info(cat_map_fit.features[1])

('Day', 31)

In [195]:
def my_init(scale):
    return lambda shape, name=None: initializations.uniform(shape, scale=scale, name=name)


In [196]:
def emb_init(shape, name=None): 
    return initializations.uniform(shape, scale=2/(shape[1]+1), name=name)

In [218]:
def get_emb(feat):
    name, c = cat_map_info(feat)
    #c2 = cat_var_dict[name]
    c2 = (c+1)//2
    if c2>50: c2=50
    inp = Input((1,), dtype='int64', name=name+'_in')
    # , W_regularizer=l2(1e-6)
    #, init=emb_init
    u = Flatten(name=name+'_flt')(Embedding(c, c2, input_length=1)(inp))
#     u = Flatten(name=name+'_flt')(Embedding(c, c2, input_length=1)(inp))
    return inp,u

In [219]:
def get_contin(feat):
    name = feat[0][0]
    inp = Input((1,), name=name+'_in')
    return inp, Dense(1, name=name+'_d',)(inp)

In [227]:
from keras.engine.input_layer import Input
from keras.layers import Dense
from keras.layers import Flatten, Dropout, Embedding
from keras import initializers
from keras.layers import merge

In [228]:
contin_inp = Input((contin_cols,), name='contin')
contin_out = Dense(contin_cols*10, activation='relu', name='contin_d')(contin_inp)
#contin_out = BatchNormalization()(contin_out)

In [233]:
embs = [get_emb(feat) for feat in cat_map_fit.features]
print(type(embs),len(embs))
print(type(embs[0]),type(embs[10]),type(embs[19]))
print(embs[0])
#conts = [get_contin(feat) for feat in contin_map_fit.features]
#contin_d = [d for inp,d in conts]
x = merge([emb for inp,emb in embs] + [contin_out], axis='concat')
#x = merge([emb for inp,emb in embs] + contin_d, mode='concat')

x = Dropout(0.02)(x)
x = Dense(1000, activation='relu')(x)
x = Dense(500, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model([inp for inp,emb in embs] + [contin_inp], x)
#model = Model([inp for inp,emb in embs] + [inp for inp,d in conts], x)
model.compile('adam', 'mean_absolute_error')
#model.compile(Adam(), 'mse')

<class 'list'> 22
<class 'tuple'> <class 'tuple'> <class 'tuple'>
(<tf.Tensor 'Store_in_13:0' shape=(?, 1) dtype=int64>, <tf.Tensor 'Store_flt_7/Reshape:0' shape=(?, ?) dtype=float32>)


TypeError: 'module' object is not callable